Импортируем нужные библиотеки.

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense

import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
# Надо импортнуть всякие керасы-херасы

In [ ]:
data_may = pd.read_excel('data_may.xlsx')
data_may.columns = data_may.iloc[1]
data_may = data_may[2:]
data_may.index = pd.RangeIndex(857)
del data_may[data_may.columns[22]]

data_july_sept = pd.read_excel('data_july-sep.xlsx')
data_july_sept.columns = data_july_sept.iloc[1]
data_july_sept = data_july_sept[2:]
data_july_sept.index = pd.RangeIndex(2881)
del data_july_sept[data_july_sept.columns[22]]

data = data_may.merge(data_july_sept, how='outer')
data

In [ ]:
i = 0
for column in data.columns:
    i += 1
    print(i*100//len(data.columns), end=' ')
    if data[column].dtypes == np.float64:
        continue
    try:
        data[column] = data[column].astype('float')
    except ValueError:
        try:
            data[column] = data[column].astype('datetime64[ns]')
        except ValueError:
            # print('First', column)
            for index in range(len(data)):
                if data.loc[index, column] is np.nan:
                    data.loc[index, column] = '0'
                data.loc[index, column] = data.loc[index, column].replace(',', '.')
            try:
                data[column] = data[column].astype('float')
            except ValueError:
                try:
                    data[column] = data[column].astype('datetime64[ns]')
                except ValueError:
                    pass
                    # print('Second', column)
data.dtypes.values

In [118]:
data_copy = data.copy()

In [119]:
columns_factors = ['Вес заготовок, т', 'Плавка в серии', 'Сечение заготовки', 'Тип разливки', 'Вес стали расчет., т', 
                   'Остаток металла в с/к, т', 'Вес разлитой стали, т', 'Остатки в п/к, т', 'Тех. обрезь, т', '№ с/к', 
                   'Темп. стали в с/к, °C', '№ п/к', 'Частота качания, кол-во/мин', 'Ход кр-ра, мм', 
                   'Скорость разливки, м/мин', 'Расход воды на кр-р, л/мин', 'Дельта температуры воды, °C', 
                   'Расход воды ЗВО №1, л/мин', 'Расход воды ЗВО №2, л/мин', 'Расход воды ЗВО №3, л/мин', 
                   'Темп 1, °С', 'Темп 2, °С', 
                   'C, %', 'Si, %', 'Mn,%', 'S, %', 'P, %', 'Cr, %', 'Ni, %', 'Cu, %', 'As, %', 'Mo, %', 'Nb, %', 
                   'Sn, %', 'Ti, %', 'V, %', 'Al, %', 'Ca, %', 'N, %', 'Pb, %', 'Mg, %', 'Zn, %', 'Сэ, %']
data_ml = data_copy[columns_factors]
data_ml

1,"Вес заготовок, т",Плавка в серии,Сечение заготовки,Тип разливки,"Вес стали расчет., т","Остаток металла в с/к, т","Вес разлитой стали, т","Остатки в п/к, т","Тех. обрезь, т",№ с/к,...,"Sn, %","Ti, %","V, %","Al, %","Ca, %","N, %","Pb, %","Mg, %","Zn, %","Сэ, %"
0,170.188,6.0,150x150,Открытая,173.088,0.4,172.688,0.0,NaN,4.0,...,0.0004,0.0007,0.0022,0.0029,0.0011,0.0040,NaN,NaN,NaN,NaN
1,144.900,4.0,150x150,Открытая,145.300,0.4,144.900,0.0,NaN,4.0,...,0.0004,0.0006,0.0030,0.0022,0.0008,0.0085,NaN,NaN,NaN,NaN
2,168.000,5.0,150x150,Открытая,168.400,0.4,168.000,0.0,NaN,7.0,...,0.0005,0.0007,0.0028,0.0031,0.0011,0.0068,NaN,NaN,NaN,NaN
3,170.100,7.0,150x150,Открытая,170.500,0.4,170.100,0.0,NaN,12.0,...,0.0006,0.0010,0.0030,0.0034,0.0005,0.0051,NaN,NaN,NaN,0.329
4,163.800,12.0,150x150,Открытая,164.200,0.4,163.800,0.0,NaN,12.0,...,0.0006,0.0010,0.0027,0.0032,0.0004,0.0038,NaN,NaN,NaN,0.318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3733,160.368,2.0,180x180,Открытая,161.121,0.4,160.721,0.0,NaN,11.0,...,NaN,0.0011,0.0016,0.0027,0.0027,0.0078,0.0047,0.0001,NaN,NaN
3734,166.536,3.0,180x180,Открытая,167.302,0.4,166.902,0.0,NaN,10.0,...,NaN,0.0012,0.0018,0.0028,0.0017,0.0069,0.0046,0.0001,NaN,NaN
3735,160.368,4.0,180x180,Открытая,161.121,0.4,160.721,0.0,NaN,1.0,...,NaN,0.0010,0.0019,0.0021,0.0017,0.0066,0.0047,NaN,NaN,NaN
3736,166.536,5.0,180x180,Открытая,167.302,0.4,166.902,0.0,NaN,3.0,...,NaN,0.0010,0.0017,0.0020,0.0015,0.0063,0.0047,NaN,NaN,NaN


In [120]:
columns_results= ['Стойкость гильзы кр-ра 1, т', 'Стойкость гильзы кр-ра 2, т', 'Стойкость гильзы кр-ра 3, т', 
                  'Стойкость гильзы кр-ра 4, т', 'Стойкость гильзы кр-ра 5, т', 'Стойкость гильзы кр-ра 6, т']
data_ml_res = data_copy[columns_results]
data_ml_res

1,"Стойкость гильзы кр-ра 1, т","Стойкость гильзы кр-ра 2, т","Стойкость гильзы кр-ра 3, т","Стойкость гильзы кр-ра 4, т","Стойкость гильзы кр-ра 5, т","Стойкость гильзы кр-ра 6, т"
0,4684.0,2925.0,1475.0,3152.0,2658.0,2487.0
1,4790.0,3091.0,1639.0,3316.0,2822.0,2646.0
2,4817.0,3118.0,1666.0,3345.0,2849.0,2676.0
3,4874.0,3175.0,1718.0,3400.0,2906.0,2730.0
4,5010.0,3315.0,1853.0,3536.0,3048.0,2869.0
...,...,...,...,...,...,...
3733,13474.0,10092.0,8088.0,8074.0,9596.0,12898.0
3734,13505.0,10116.0,8113.0,8098.0,9627.0,12929.0
3735,13530.0,10144.0,8140.0,8126.0,9651.0,12957.0
3736,13557.0,10172.0,8168.0,8157.0,9679.0,12981.0


In [121]:
data_ml_res.count()

1
Стойкость гильзы кр-ра 1, т    3730
Стойкость гильзы кр-ра 2, т    3635
Стойкость гильзы кр-ра 3, т    3634
Стойкость гильзы кр-ра 4, т    3673
Стойкость гильзы кр-ра 5, т    3718
Стойкость гильзы кр-ра 6, т    3730
dtype: int64

In [122]:
data_ml.fillna(0, inplace=True)

In [123]:
data_ml.loc[data_ml['Тип разливки'] == 'Закрытая', 'Тип разливки'] = 1
data_ml.loc[data_ml['Тип разливки'] == 'Открытая', 'Тип разливки'] = 0
data_ml.loc[data_ml['Сечение заготовки'] == '150x150', 'Сечение заготовки'] = 150
data_ml.loc[data_ml['Сечение заготовки'] == '180x180', 'Сечение заготовки'] = 180
data_ml.loc[:, 'Сечение заготовки'] = data_ml['Сечение заготовки'].astype('float')
data_ml.loc[:, 'Тип разливки'] = data_ml['Тип разливки'].astype('float')
data_ml.dtypes.values

array([dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64'), dtype('float64'), dtype('float64'),
       dtype('float64')], dtype=object)

In [124]:
data_ml[data_ml_res.columns] = data_ml_res
data_ml = data_ml.dropna()
data_ml

1,"Вес заготовок, т",Плавка в серии,Сечение заготовки,Тип разливки,"Вес стали расчет., т","Остаток металла в с/к, т","Вес разлитой стали, т","Остатки в п/к, т","Тех. обрезь, т",№ с/к,...,"Pb, %","Mg, %","Zn, %","Сэ, %","Стойкость гильзы кр-ра 1, т","Стойкость гильзы кр-ра 2, т","Стойкость гильзы кр-ра 3, т","Стойкость гильзы кр-ра 4, т","Стойкость гильзы кр-ра 5, т","Стойкость гильзы кр-ра 6, т"
0,170.188,6.0,150.0,0.0,173.088,0.4,172.688,0.0,0.0,4.0,...,0.0000,0.0000,0.0,0.000,4684.0,2925.0,1475.0,3152.0,2658.0,2487.0
1,144.900,4.0,150.0,0.0,145.300,0.4,144.900,0.0,0.0,4.0,...,0.0000,0.0000,0.0,0.000,4790.0,3091.0,1639.0,3316.0,2822.0,2646.0
2,168.000,5.0,150.0,0.0,168.400,0.4,168.000,0.0,0.0,7.0,...,0.0000,0.0000,0.0,0.000,4817.0,3118.0,1666.0,3345.0,2849.0,2676.0
3,170.100,7.0,150.0,0.0,170.500,0.4,170.100,0.0,0.0,12.0,...,0.0000,0.0000,0.0,0.329,4874.0,3175.0,1718.0,3400.0,2906.0,2730.0
4,163.800,12.0,150.0,0.0,164.200,0.4,163.800,0.0,0.0,12.0,...,0.0000,0.0000,0.0,0.318,5010.0,3315.0,1853.0,3536.0,3048.0,2869.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3733,160.368,2.0,180.0,0.0,161.121,0.4,160.721,0.0,0.0,11.0,...,0.0047,0.0001,0.0,0.000,13474.0,10092.0,8088.0,8074.0,9596.0,12898.0
3734,166.536,3.0,180.0,0.0,167.302,0.4,166.902,0.0,0.0,10.0,...,0.0046,0.0001,0.0,0.000,13505.0,10116.0,8113.0,8098.0,9627.0,12929.0
3735,160.368,4.0,180.0,0.0,161.121,0.4,160.721,0.0,0.0,1.0,...,0.0047,0.0000,0.0,0.000,13530.0,10144.0,8140.0,8126.0,9651.0,12957.0
3736,166.536,5.0,180.0,0.0,167.302,0.4,166.902,0.0,0.0,3.0,...,0.0047,0.0000,0.0,0.000,13557.0,10172.0,8168.0,8157.0,9679.0,12981.0


In [125]:
data_ml['Стойкость гильзы кр-ра 1, т'].max()

17731.0

In [175]:
model = keras.Sequential([
    keras.layers.Input(1, shape=(43)),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(43, activation='linear')
])

TypeError: Input() got multiple values for argument 'shape'

In [176]:
fashion_mnist = keras.datasets.fashion_mnist
(train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()

In [177]:
train_y.size

60000

In [178]:
wall = len(data_ml.index)*0.7

train_x1 = data_ml.loc[:wall, columns_factors].to_numpy()
train_y1 = data_ml.loc[:wall, columns_results].to_numpy()

test_x1 = data_ml.loc[wall:, columns_factors].to_numpy()
test_y1 = data_ml.loc[wall:, columns_results].to_numpy()
train_y1 = train_y1[:, 0]
test_y1 = test_y1[:, 0]
train_y1

array([4684., 4790., 4817., ..., 1313., 1341., 1368.])

In [179]:
train_x1

array([[1.70188e+02, 6.00000e+00, 1.50000e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.44900e+02, 4.00000e+00, 1.50000e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.68000e+02, 5.00000e+00, 1.50000e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [1.62816e+02, 1.00000e+01, 1.80000e+02, ..., 1.00000e-04,
        1.80000e-03, 0.00000e+00],
       [1.65888e+02, 1.10000e+01, 1.80000e+02, ..., 1.00000e-04,
        4.60000e-03, 0.00000e+00],
       [1.65888e+02, 1.20000e+01, 1.80000e+02, ..., 1.00000e-04,
        1.50000e-03, 0.00000e+00]])

In [180]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_absolute_percentage_error'])

In [181]:
history = model.fit(train_x1, train_y1, batch_size=64, epochs=50)
test_scores = model.evaluate(test_x1, test_y1)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

ValueError: A target array with shape (2318, 1) was passed for an output of shape (None, 43) while using as loss `mean_squared_error`. This loss expects targets to have the same shape as the output.

In [156]:
test_loss, test_acc = model.evaluate(x=test_x1,  y=test_y1, verbose=1)
test_loss, test_acc

ValueError: A target array with shape (1133, 1) was passed for an output of shape (None, 17732) while using as loss `mean_squared_error`. This loss expects targets to have the same shape as the output.

In [185]:
model1=Sequential()
model1.add(Dense(64,input_dim=43,activation='relu'))
model1.add(Dense(20,activation='relu'))
model1.add(Dense(1,activation='linear'))

In [186]:
model1.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_percentage_error'])

In [187]:
model1.fit(train_x1, train_y1, batch_size=64, epochs=50, validation_split=0.2)

Train on 1854 samples, validate on 464 samples
Epoch 1/50
1854/1854 [==============================] - 0s 226us/step - loss: 22789506.1909 - mean_absolute_percentage_error: 104.8256 - val_loss: 13672754.3707 - val_mean_absolute_percentage_error: 132.6347
Epoch 2/50
1854/1854 [==============================] - 0s 27us/step - loss: 10331684.7864 - mean_absolute_percentage_error: 157.4866 - val_loss: 10078691.2155 - val_mean_absolute_percentage_error: 244.3029
Epoch 3/50
1854/1854 [==============================] - 0s 23us/step - loss: 8087989.7632 - mean_absolute_percentage_error: 213.3487 - val_loss: 10122119.5991 - val_mean_absolute_percentage_error: 246.0480
Epoch 4/50
1854/1854 [==============================] - 0s 24us/step - loss: 7988546.5210 - mean_absolute_percentage_error: 200.6931 - val_loss: 10075100.2284 - val_mean_absolute_percentage_error: 244.1984
Epoch 5/50
1854/1854 [==============================] - 0s 25us/step - loss: 7973700.3463 - mean_absolute_percentage_error: 20

In [188]:
test_scores = model1.evaluate(test_x1, test_y1)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

1133/1133 [==============================] - 0s 22us/step
Test loss: 16559440.5863857
Test accuracy: 122.97723388671875
